In [1]:
import pandas as pd
import glob
import numpy as np
import json
from tld import get_tld
from pandarallel import pandarallel
pandarallel.initialize()

datafiles = glob.glob("/home/phadke/ONR/ONR/big_data/Twitter/*.csv")

linkframe = pd.DataFrame(columns=['party','link', 'author'])

counter = 0

datafiles[0]

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


'/home/phadke/ONR/ONR/big_data/Twitter/4636722562_Thakur_deepak_s_INC.csv'

In [2]:
multiparty = ['_AAP.csv']
valid_files = []

for d in datafiles:
    for m in multiparty:
        if m in d:
            valid_files.append(d)
            

In [3]:
print(len(datafiles))
print(len(valid_files))

6691
430


In [4]:
for d in valid_files:
    if counter%100==0:
        print(counter)
    counter+=1
    splitfilename = d.replace(".csv", "").split("_")
    party = splitfilename[len(splitfilename)-1]
    author = splitfilename[len(splitfilename)-2]
    with open(d, "r") as jsonfile:

        for line in jsonfile:
            job= json.loads(line)
            if "entities" in job.keys():
                if "urls" in job["entities"]:
                    for u in job["entities"]["urls"]:
                        if "expanded_url" in u:
                            linkframe = linkframe.append({"party":party, "link":u["expanded_url"], "author":author}, ignore_index=True)


0
100
200
300
400


In [5]:
linkframe.head()

,party,link,author
0,AAP,https://twitter.com/gill_buta/status/144960713...,buta
1,AAP,https://twitter.com/gill_buta/status/144354531...,buta
2,AAP,https://twitter.com/gill_buta/status/125759475...,buta
3,AAP,https://twitter.com/gill_buta/status/125697840...,buta
4,AAP,https://twitter.com/gill_buta/status/125614398...,buta


In [6]:
#print(linkframe.head())

linkframe.to_csv("/home/phadke/ONR/ONR/lite_data/aap_dec15_nonfiltered_extracted_urls.csv")
## extract domains

def get_domain(link):
    try:
        res = get_tld(link, as_object=True)
        return res.fld
    except:
        fakevar=1


linkframe['domain'] = linkframe['link'].parallel_apply(lambda x: get_domain(x))



In [7]:
#print(linkframe.head())

common_domains = ['twitter.com', 'facebook.com', 'google.com', 'm.tech', 'm.sc', 'b.tech', 'page.link', 'youtu.be', 'bit.ly', 'instagram.com','youtube.com']

filtered_link = linkframe.loc[~linkframe['domain'].isin(common_domains)]

filtered_link.to_csv("/home/phadke/ONR/ONR/lite_data/aap_dec15_filtered_extracted_urls.csv")

agg_filtered = filtered_link.groupby(['author','domain']).size().reset_index().rename(columns={0:"count"})

print(agg_filtered.head())

agg_filtered.to_csv("/home/phadke/ONR/ONR/lite_data/aap_dec15_extracted_domains.csv")

  author             domain  count
0                      1.si      1
1            54.xn--h2brj9c      1
2           5dariyanews.com      1
3                 aajtak.in      8
4         aamaadmiparty.org      1


In [8]:
agg_filtered.sample(5)

,author,domain,count
3157,KuldeepKumarAAP,punjabkesari.in,1
2246,Aojhaaap,janchowk.com,1
2876,ElvinBaruah,newsijt.com,1
3331,Mishra,outlookindia.com,1
2785,DrKumarVishwas,hindustantimes.com,1


In [9]:
len(agg_filtered)

6342